# Classes and OOP

Classes are the mechanism used to create new kinds of objects.

## The `class` Statement

A `class` defines a set of attributes that are **associated with**, and **shared by**, a collection of objects known as **instances**. A class is most commonly a collection of functions (known as **methods**), variables (which are known as **class variables**), and computed attributes (which are known as **properties**).

In [2]:
class Account(object):
    # class variable, shared by all instances.
    num_accounts = 0

    def __init__(self, name, balance):
        self.name = name
        self.balance = balance
        Account.num_accounts += 1

    def __del__(self):
        Account.num_accounts -= 1

    # instance method.
    def deposit(self, amt):
        self.balance += amt

    def withdraw(self, amt):
        self.balance -= amt

    def inquiry(self):
        return self.balance
    
a = Account('Guido', 1000.00)

print(a.inquiry())
a.deposit(100)
print(a.inquiry())
print(a.name)

1000.0
1100.0
Guido


When you access an attribute, the instance is checked first and if nothing is known, the search moves to the instance’s class instead. This is the underlying mechanism by which a class shares its attributes with all of its instances.

## Scoping Rules

Although classes define a namespace, classes do not create a scope for names used inside the bodies of methods. Therefore, when you’re implementing a class, references to attributes and methods must be fully qualified. For example, in methods you always reference attributes of the instance through self.

The explicit use of self is required because **Python does not provide a means to explicitly declare variables** (that is, a declaration such as `int x` or `float y` in C). Without this, there is no way to know whether an assignment to a variable in a method is supposed to be a local variable or if it’s supposed to be saved as an instance attribute.

## Inheritance

*Inheritance* is a mechanism for creating a new class that specializes or modifies the behavior of an existing class.The original class is called a *base class* or a superclass.The new class is called a *derived class* or a subclass.



In [4]:
import random

class EvilAccount(Account):
    def inquiry(self):
        if random.randint(0, 4) == 1:
            return self.balance * 1.10
        else:
            return self.balance

c = EvilAccount("George", 1000.0)
c.deposit(10.0)
print(c.inquiry())

1010.0


A subclass can add new attributes to the instances by defining its own version of `__init__()`. When a derived class defines `__init__()`, the `__init__()` methods of base classes are not automatically invoked. Therefore, it’s up to a derived class to perform the proper initialization of the base classes by calling their `__init__()` methods.

If you don’t know whether the base class defines `__init__()`, it is always safe to call it without any arguments because
there is always a default implementation that simply does nothing.

Python supports multiple inheritance.This is specified by having a class list multiple base classes.

### Mixin

See more: [Mixin](https://en.wikipedia.org/wiki/Mixin)

## Polymorphism Dynamic Binding and Duck Typing

*Dynamic binding* (also sometimes referred to as polymorphism when used in the context of inheritance) is the capability to use an instance without regard for its type. It is handled entirely through the attribute lookup process described for inheritance in the preceding section.

A critical aspect of this binding process is that *it is independent of what kind of object obj is*. Thus, if you make a lookup such as obj.name, it will work on any obj that happens to have a name attribute. This behavior is sometimes referred to as duck typing in reference to the **adage “if it looks like, quacks like, and walks like a duck, then it’s a duck.”**

This latter approach is often used to maintain a loose coupling of program components. One of the most common
examples is with various “file-like” objects defined in the standard library. Although these objects work like files, they don’t inherit from the built-in file object.

## Static Methods and Class Methods

In a class definition, all functions are assumed to operate on an instance, which is always passed as the first parameter self. However, there are two other common kinds of methods that can be defined.

A static method is an ordinary function that just happens to live in the namespace defined by a class. It does not operate on any kind of instance.

In [5]:
import time

class Date():

    def __init__(self, year, month, day):
        self.year = year
        self.month = month
        self.day = day

    def __str__(self):
        return '%d/%d/%d' % (self.month, self.day, self.year)

    @staticmethod
    def now():
        t = time.localtime()
        return Date(t.tm_year, t.tm_mon, t.tm_mday)

    @staticmethod
    def tomorrow():
        t = time.localtime(time.time() + 86400)
        return Date(t.tm_year, t.tm_mon, t.tm_mday)


print(Date(1970, 1, 15))
print(Date.now())
print(Date.tomorrow())

1/15/1970
8/11/2015
8/12/2015


A common use of static methods is in writing classes where you might have many different ways to create new instances. Because there can only be one `__init__()` function, alternative creation functions are often defined by static methods.

Class methods are methods that operate on the class itself as an object. Defined using the `@classmethod` decorator, a class method is different than an instance method in that the class is passed as the first argument which is named cls by convention.

In [6]:
class Times(object):
    factor = 1

    @classmethod
    def mul(cls, x):
        return cls.factor * x


class TwoTimes(Times):
    factor = 2

TwoTimes.mul(3)

6

## Properties

Normally, when you access an attribute of an instance or a class, the associated value that is stored is returned. A property is *a special kind of attribute* that computes its value when accessed.



In [7]:
import math

class Circle(object):

    def __init__(self, radius):
        self.radius = radius

    @property
    def area(self):
        return math.pi * self.radius ** 2

    @property
    def perimeter(self):
        return 2 * math.pi * self.radius


c = Circle(4.0)
print(c.radius)
print(c.area)
print(c.perimeter)
c.area = 2

4.0
50.2654824574
25.1327412287


AttributeError: can't set attribute

With properties, we don't need `()` explicitly. Using properties in this way is related to something known as the *Uniform Access Principle*. So c.radius and c.aear look similar.

Properties can also **intercept operations** to set and delete an attribute. This is done by attaching additional setter and deleter methods to a property.

In [8]:
class Foo(object):
    def __init__(self, name):
        self.__name = name

    @property
    def name(self):
        return self.__name

    @name.setter
    def name(self, value):
        if not isinstance(value, str):
            raise TypeError('Must be a string.')
        self.__name = value

    @name.deleter
    def name(self):
        raise TypeError('You are not allowed to delete name property.')

f = Foo("Guido")
print(f.name)
f.name = "Monty"
f.name = 42
del f.name

Guido


TypeError: Must be a string.

## Descriptors

With properties, access to an attribute is controlled by a series of user-defined get, set, and delete functions.This sort of attribute control can be further generalized through the use of a descriptor object.A descriptor is simply an object that represents the value of an attribute.

In [9]:
class TypeProperty(object):

    def __init__(self, name, type, default=None):
        self.name = '_' + name
        self.type = type

        self.default = default if default else type()

    def __get__(self, instance, owner):
        return getattr(instance, self.name, self.default)

    def __set__(self, instance, value):
        if not isinstance(value, self.type):
            raise TypeError('Must be a %s' % self.type)
        setattr(instance, self.name, value)

    def __delete__(self, instance):
        raise AttributeError('Cannot delete attribute')


class Foo(object):
    name = TypeProperty("name", str)
    num = TypeProperty("num", int, 42)


f = Foo()
a = f.name
f.name = "Guido"
del f.name

AttributeError: Cannot delete attribute

## Data Encapsulation and Private Attributes

By default, all attributes and methods of a class are “public.”This means that they are all accessible without any restrictions. It also implies that everything defined in a base class is inherited and accessible within a derived class.

To fix this problem, all names in a class that start with a double underscore, such as `__Foo`, are automatically mangled to form a new name of the form `_Classname__Foo`.

In [10]:
class A(object):
    def __init__(self):
        self.__X = 3

    def __spam(self):
        pass

    def bar(self):
        self.__spam()


class B(A):
    def __init__(self):
        A.__init__(self)
        self.__X = 37

    def __spam(self):
        pass

a = A()
print(a._A__X)
print(a.__X)

3


AttributeError: 'A' object has no attribute '__X'

It is recommended that private attributes be used when defining mutable attributes via properties. By doing so, you will encourage users to use the property name rather than accessing the underlying instance data directly (which is probably not what you intended if you wrapped it with a property to begin with).

Finally, don’t confuse the naming of private class attributes with the naming of “private” definitions in a module.A common mistake is to define a class where a single leading underscore is used on attribute names in an effort to hide their values (e.g., `_name`). In modules, this naming convention prevents names from being exported by the `from module import *` statement. However, in classes, this naming convention does not hide the attribute nor does it prevent name clashes that arise if someone inherits from the class and defines a new attribute or method with the same name.

## Object Memory Management

When a class is defined, the resulting class is a factory for creating new instances.

The creation of an instance is carried out in two steps using the special method `__new__()`, which creates a new instance, and `__init__()`, which initializes it.

If you see `__new__()` defined in a class, it usually means the class is doing one of two things. First, the class might be inheriting from a base class whose instances are immutable.This is common if defining objects that inherit from an immutable built-in type such as an integer, string, or tuple. The other major use of `__new__()` is when defining metaclasses.

Once created, instances are managed by reference counting. If the reference count reaches zero, the instance is immediately destroyed.When the instance is about to be destroyed, the interpreter first looks for a `__del__()` method associated with the object and calls it. (A better approach may be to define a method such as `close()` that a program can use to explicitly perform a shutdown.)

## Object Representation and Attribute Binding

Internally, instances are implemented using a dictionary that’s accessible as the instance’s `__dict__` attribute. This dictionary contains the data that’s unique to each instance.

In [4]:
class User(object):
    def __init__(self, id, name, hobby):
        self.id = id
        self.name = name
        self.hobby = hobby
        
    
u = User(1, 'anders', 'python')
print u.__dict__

# add a new attribute
u.number = 123456789
print u.__dict__

# add a new attribute to __dict__
u.__dict__['city'] = 'shanghai'
print u.city

{'hobby': 'python', 'id': 1, 'name': 'anders'}
{'hobby': 'python', 'number': 123456789, 'id': 1, 'name': 'anders'}
shanghai


In [7]:
# Instances are linked back to their class by a special attribute __class__.
print u.__class__

print User.__dict__.keys()

print User.__bases__

<class '__main__.User'>
['__dict__', '__module__', '__weakref__', '__doc__', '__init__']
(<type 'object'>,)


Instances are linked back to their class by a special attribute `__class__`. The class itself is also just a thin layer over a dictionary which can be found in its own `__dict__` attribute.The class dictionary is where you find the methods. Classes are linked to their base classes in a special attribute `__bases__`, which is a tuple of the base classes. This underlying structure is the basis for all of the operations that get, set, and delete the attributes of objects.

Whenever an attribute is set using obj.name = value, the special method `obj.__setattr__("name", value)` is invoked. If an attribute is deleted using del obj.name, the special method `obj.__delattr__("name")` is invoked.The default behavior of these methods is to modify or remove values from the local `__dict__` of obj unless the requested attribute happens to correspond to a property or descriptor.

For attribute lookup such as obj.name, the special method `obj.__getattrribute__("name")` is invoked. This method carries out the search process for finding the attribute, which normally includes checking for properties, looking in the local `__dict__` attribute, checking the class dictionary, and searching the base classes. If this search process fails, a final attempt to find the attribute is made by trying to invoke the `__getattr__()` method of the class (if defined). If this fails, an `AttributeError` exception is raised.

User-defined classes can implement their own versions of the attribute access func- tions, if desired.

In [10]:
import math

class Circle(object):
    def __init__(self, radius):
        self.radius = radius
        
    def __getattr__(self, name):
        if name == 'area':
            return math.pi * self.radius ** 2
        elif name == 'perimeter':
            return 2 * math.pi * self.radius
        else:
            return object.__getattr__(self, name)
    
    def __setattr__(self, name, value):
        if name in ['area', 'perimeter']:
            raise TypeError('%s is readonly' % name)
        object.__setattr__(self, name, value)
        
c = Circle(3)
print c.radius
print c.area
print c.perimeter
# print c.something  # AttributeError

c.radius = 5
print c.area
c.area = 10

3
28.2743338823
18.8495559215
78.5398163397


TypeError: area is readonly

A class that reimplements these methods should probably rely upon the default implementation in object to carry out the actual work. This is because the default implementation takes care of the more advanced features of classes such as **descriptors and properties**.

## `__slots__`

A class can restrict the set of legal instance attribute names by defining a special variable called `__slots__`. When `__slots__` is defined, the attribute names that can be assigned on instances are restricted to the names specified. Otherwise, an AttributeError exception is raised. In reality, `__slots__` was never implemented to be a safety feature. Instead, it is actually a **performance optimization** for both memory and execution speed. Instances of a class that uses `__slots__` no longer use a dictionary for storing instance data. Instead, a much more compact data structure based on an array is used.

## Operator Overloading

User-defined objects can be made to work with all of Python’s built-in operators by adding implementations of the corresponding special methods to a class. For example, if you wanted to add a new kind of number to Python, you could define a class in which special methods such as `__add__()` were defined to make instances work with the standard mathematical operators.

In [21]:
import math

class Complex(object):
    def __init__(self, real, imag=0):
        self.real = real
        self.imag = imag
        
    def __repr__(self):
        return 'Complex(%s, %s)' % (self.real, self.imag)
    
    def __str__(self):
        sign = '+' if self.imag >= 0 else '-'
        return '(%g %s %gj)' % (self.real, sign, abs(self.imag))
    
    # self + other
    def __add__(self, other):
        return Complex(self.real+other.real, self.imag+other.imag)
    
    # self - other
    def __sub__(self, other):
        return Complex(self.real-other.real, self.imag-other.imag)
    
    def __radd__(self, other):
        return Complex(self.real+other.real, self.imag+other.imag)
    
    def __rsub__(self, other):
        return Complex(other.real-self.real, other.imag-self.imag)
    
c1 = Complex(1, 2)
print c1

c2 = Complex(2, 3)
print repr(c2)

print(c1+c2)
print(c1-c2)

print(1.0 + c1)

(1 + 2j)
Complex(2, 3)
(3 + 5j)
(-1 - 1j)
(2 + 2j)


## Types and Class Membership Tests

Use `isinstance`, `issubclass`, function

In [27]:
class A(object): pass
class B(A): pass
class C(object): pass

a = A()
b = B()
c = C()

print type(a)
print '--------------'
print isinstance(a, A)
print isinstance(b, A)
print isinstance(b, B)
print isinstance(b, C)

# sub class
print '--------------'
print issubclass(A, A)
print issubclass(B, A)
print issubclass(C, A)



<class '__main__.A'>
--------------
True
True
True
False
--------------
True
True
False


For duck type case, pay attention to `__instancecheck` and `subclasscheck`.

## Abstract Base Classes

We know that the `isinstance()` and `issubclass()` operations can be overloaded. This can be used to create objects that group similar classes together and to perform various forms of type-checking. Abstract base classes build upon this concept and provide a means for organizing objects into a **hierarchy**, making assertions about required methods, and so forth.

In [31]:
from abc import ABCMeta, abstractmethod, abstractproperty
class Foo:
    __metaclass__ = ABCMeta
    @abstractmethod
    def spam(self, a, b):
        pass
    @abstractproperty
    def name(selff):
        pass
    
class Grok(object):
    def spam(self, a, b):
        print 'Grok.spam'

Foo.register(Grok)
g = Grok()
print isinstance(g, Foo)

True


## Metaclasses

When you define a class in Python, the class definition itself becomes an object.

In [32]:
class Foo(object): pass
isinstance(Foo, object)

True

If you think about this long enough, you will realize that something had to create the Foo object.This creation of the class object is controlled by a special kind of object called a metaclass. Simply stated, a metaclass is an object that knows how to create and manage classes.